# Harnessing Weather Insights for Accurate Energy Load Forecasting

In [1]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


### Import important libraries

In [2]:
import platform
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.linalg import DenseVector
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

from pathlib import Path

In [3]:
# Initialize via the full spark path
if platform.system() == 'Windows':
    print("Windows OS detected")
    findspark.init("C:/Spark/spark-3.5.4-bin-hadoop3") # For my local machine
else:
    findspark.init("/usr/local/spark/")

Windows OS detected


In [4]:
# Build the SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Linear Regression Model") \
    .config("spark.executor.memory", "16g") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.15.0") \
    .config("spark.executor.heartbeatInterval", "100s") \
    .config("spark.sql.session.timeZone", "UTC") \
    .getOrCreate()

   
# Main entry point for Spark functionality. A SparkContext represents the
# connection to a Spark cluster, and can be used to create :class:`RDD` and
# broadcast variables on that cluster.      
sc = spark.sparkContext

### Preprocessing I: Read in Weather and Load Data

In [5]:
# Read in the data

# Folder Structure
# data
# |-- geosphere
# |   |-- YYYY
# |      |-- MM.csv
# |      |-- MM.csv
# |
# |-- transparency
# |   |-- YYYY
# |      |-- MM.xml
# |      |-- MM.xml

# Loop through the geosphere folder and read in the data

# Define the base data folder
base_path = Path("./data/geosphere")

# Collect all data frames first to optimize the union operation
dfs = []

for year_folder in base_path.iterdir():
    if year_folder.is_dir():
        for month_file in year_folder.glob("*.csv"):
            print(f"Reading in {month_file}")

            df = spark.read.csv(str(month_file), header=True, inferSchema=True)

            # Convert the time column (string) to a timestamp
            df = df.withColumn("time", to_timestamp(col("time"), "yyyy-MM-dd'T'HH:mmXXX"))
            
            dfs.append(df)
            
            print(f"Read in {df.count()} rows")

if dfs:
    # Combine all DataFrames
    weather = dfs[0]
    for df in dfs[1:]:
        weather = weather.union(df)

    # Aggregate measurements (average from different stations)
    weather = (
        weather.groupBy("time")
        .agg(
            avg("rr").alias("avg_rr"),
            avg("tl").alias("avg_tl"),
            avg("p").alias("avg_p"),
            avg("so_h").alias("avg_so_h"),
            avg("ff").alias("avg_ff"),
        )
        .orderBy("time")
    )

    weather.show(10, truncate=False)
    
    print(weather.count())
    weather.printSchema()
else:
    print("No data found")


Reading in data\geosphere\2022\01.csv
Read in 1490 rows
Reading in data\geosphere\2022\02.csv
Read in 1346 rows
Reading in data\geosphere\2022\03.csv
Read in 1490 rows
Reading in data\geosphere\2022\04.csv
Read in 1442 rows
Reading in data\geosphere\2022\05.csv
Read in 1490 rows
Reading in data\geosphere\2022\06.csv
Read in 1442 rows
Reading in data\geosphere\2022\07.csv
Read in 1490 rows
Reading in data\geosphere\2022\08.csv
Read in 1490 rows
Reading in data\geosphere\2022\09.csv
Read in 1442 rows
Reading in data\geosphere\2022\10.csv
Read in 1490 rows
Reading in data\geosphere\2022\11.csv
Read in 1442 rows
Reading in data\geosphere\2022\12.csv
Read in 1490 rows
Reading in data\geosphere\2023\01.csv
Read in 1490 rows
Reading in data\geosphere\2023\02.csv
Read in 1346 rows
Reading in data\geosphere\2023\03.csv
Read in 1490 rows
Reading in data\geosphere\2023\04.csv
Read in 1442 rows
Reading in data\geosphere\2023\05.csv
Read in 1490 rows
Reading in data\geosphere\2023\06.csv
Read in 14

In [6]:
# Loop through the transparency folder and read in the energy data

# Define base path for transparency data
base_path = Path("./data/transparency")

# Collect DataFrames before performing union (optimization)
dfs = []

for year_folder in base_path.iterdir():
    if year_folder.is_dir():
        for month_file in year_folder.glob("*.xml"):
            print(f"Reading transparency data: {month_file}")

            # Read XML data
            df = spark.read.format('xml').option("rowTag", "GL_MarketDocument").load(str(month_file))

            # Extract and explode relevant fields
            df_filtered = df.select(
                col("TimeSeries.Period.timeInterval.start").alias("start_time"),
                col("TimeSeries.Period.timeInterval.end").alias("end_time"),
                col("TimeSeries.Period.resolution").alias("resolution"),
                explode(col("TimeSeries.Period.Point")).alias("Point")  # Flatten Points
            ).select(
                col("start_time"),
                col("end_time"),
                col("resolution"),
                col("Point.position").cast("int").alias("position"),
                col("Point.quantity").cast("double").alias("quantity")
            )

            # Convert ISO 8601 duration (e.g., "PT15M") to minutes dynamically
            df_fixed = df_filtered.withColumn(
                "interval_minutes",
                expr("CAST(SUBSTRING(resolution, 3, LENGTH(resolution) - 3) AS INT)")  # Extracts "15" from "PT15M"
            ).withColumn(
                "actual_time",
                expr("start_time + (position - 1) * interval_minutes * interval 1 minute")
            ).select(
                col("actual_time"),
                col("quantity")
            )
            
            # Aggregate to hourly intervals
            df_hourly = df_fixed.withColumn(
                "hourly_time", date_trunc("hour", col("actual_time"))  # Round down to the hour
            ).groupBy("hourly_time").agg(
                sum("quantity").alias("hourly_quantity")  # Sum all sub-hourly measurements
            )

            # Rename to match the structure of other time series
            df_hourly = df_hourly.select(
                col("hourly_time").alias("timestamp"), col("hourly_quantity").alias("quantity")
            ).orderBy("timestamp")
            
            print(f"Read in {df_hourly.count()} values")
            
            # Append DataFrame to list
            dfs.append(df_hourly)

# Merge all collected DataFrames
if dfs:
    Load = dfs[0]
    for df in dfs[1:]:
        Load = Load.union(df)

    Load.show(10)
    Load.printSchema()
else:
    print("No data found.")



Reading transparency data: data\transparency\2022\01.xml
Read in 744 values
Reading transparency data: data\transparency\2022\02.xml
Read in 672 values
Reading transparency data: data\transparency\2022\03.xml
Read in 744 values
Reading transparency data: data\transparency\2022\04.xml
Read in 720 values
Reading transparency data: data\transparency\2022\05.xml
Read in 744 values
Reading transparency data: data\transparency\2022\06.xml
Read in 720 values
Reading transparency data: data\transparency\2022\07.xml
Read in 744 values
Reading transparency data: data\transparency\2022\08.xml
Read in 744 values
Reading transparency data: data\transparency\2022\09.xml
Read in 720 values
Reading transparency data: data\transparency\2022\10.xml
Read in 744 values
Reading transparency data: data\transparency\2022\11.xml
Read in 720 values
Reading transparency data: data\transparency\2022\12.xml
Read in 744 values
Reading transparency data: data\transparency\2023\01.xml
Read in 744 values
Reading tran

### Preprocessing II: Combine both Data Frames

In [7]:
if Load is not None and weather is not None:
    # Join the data into a single DataFrame
    data = Load.join(weather, Load.timestamp == weather.time, "inner").drop("time")
    
    # Rename columns for better understanding
    data = data.withColumnRenamed("timestamp", "time")
    data = data.withColumnRenamed("quantity", "load")

    data = data.withColumnRenamed("avg_rr", "rainfall")
    data = data.withColumnRenamed("avg_tl", "temperature")
    data = data.withColumnRenamed("avg_p", "pressure")
    data = data.withColumnRenamed("avg_so_h", "sunshine_duration")
    data = data.withColumnRenamed("avg_ff", "wind_speed")
    
    # Reorder the columns
    data = data.select("load", "time", "rainfall", "temperature", "pressure", "sunshine_duration", "wind_speed")
    
    # Print the schema and stats
    data.describe().show()
    data.show(10)
    data.printSchema()

+-------+-----------------+-------------------+------------------+-----------------+------------------+------------------+
|summary|             load|           rainfall|       temperature|         pressure| sunshine_duration|        wind_speed|
+-------+-----------------+-------------------+------------------+-----------------+------------------+------------------+
|  count|            26304|              26304|             26304|            26304|             26304|             26304|
|   mean|27147.53615419708|0.09834435827250564|10.736714948296894|959.9205348996387|0.2120685066909985| 2.136152296228693|
| stddev|5198.551548542342| 0.6008211250333243|   8.2578055043952|7.875779934448332|0.3395092943434039|1.3306188964835226|
|    min|          16055.0|                0.0|             -11.0|            929.4|               0.0|              0.05|
|    max|          41273.0|               46.3|              33.8|            982.8|               1.0|             10.35|
+-------+-------

### Machine Learning

In [8]:
# Keep original timestamp before converting to Unix seconds
unix_time_data = data.withColumn("unix_time", unix_timestamp("time"))  # Store Unix time separately

# Extract day of the year (1-365/366) from the original timestamp column
unix_time_data = unix_time_data.withColumn("day_of_year", dayofyear(col("time")))

# Extract time of day in seconds (seconds since midnight)
unix_time_data = unix_time_data.withColumn(
    "time_of_day", expr("hour(time) * 3600 + minute(time) * 60 + second(time)")
)

# Normalize day of the year to range (0, 2π) for periodic encoding
ml_data = unix_time_data.withColumn("day_sin", sin(2 * 3.1416 * col("day_of_year") / 365))
ml_data = ml_data.withColumn("day_cos", cos(2 * 3.1416 * col("day_of_year") / 365))

# Normalize time of day
ml_data = ml_data.withColumn("time_sin", sin(2 * 3.1416 * col("time_of_day") / 86400))
ml_data = ml_data.withColumn("time_cos", cos(2 * 3.1416 * col("time_of_day") / 86400))

# Add weekend feature
ml_data = ml_data.withColumn("is_weekend", when(col("time").cast("string").like("%Sat%") | col("time").cast("string").like("%Sun%"), lit(1)).otherwise(lit(0)))

# Define new feature columns
feature_columns = ["day_sin", "day_cos", "time_sin", "time_cos", "is_weekend", "rainfall", "temperature", "pressure", "sunshine_duration", "wind_speed"]


assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

# Transform data
df_ml = assembler.transform(ml_data).select("load", "features")

df_ml.cache()

# Show transformed data
df_ml.show(5, truncate=False)



+-------+-----------------------------------------------------------------------------------------------------------------------------------------+
|load   |features                                                                                                                                 |
+-------+-----------------------------------------------------------------------------------------------------------------------------------------+
|20965.0|[0.017213396404173065,0.9998518385162037,0.8660266281835433,0.4999978792725455,0.0,0.0,11.95,965.8,0.0,3.75]                             |
|27933.0|[0.034421692083641306,0.9994073979684656,0.4999946981757423,-0.8660284647724625,0.0,0.0,2.6999999999999997,964.4,0.55,0.8500000000000001]|
|26437.0|[0.034421692083641306,0.9994073979684656,-7.346410206643587E-6,-0.9999999999730151,0.0,0.0,6.1,962.65,0.95,0.75]                         |
|25867.0|[0.034421692083641306,0.9994073979684656,-0.2588267325222856,-0.9659237664183635,0.0,0.0,7.25,962.09999

In [9]:
scaler = StandardScaler(inputCol="features", outputCol="features_scaled")
scaled_df = scaler.fit(df_ml).transform(df_ml).select("load", "features_scaled")
scaled_df = scaled_df.withColumnRenamed("features_scaled", "features")
scaled_df.cache()

scaled_df.show(5, truncate=False)

+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|load   |features                                                                                                                                                            |
+-------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|20965.0|[0.024354090758037714,1.4133324448072522,1.2247247212238557,0.7070895331403335,0.0,0.0,1.4471157008529243,122.62912473920099,0.0,2.8182374456805577]                |
|27933.0|[0.048700964839629,1.4127042094810678,0.7070866500041507,-1.2247245202982293,0.0,0.0,0.3269633801090289,122.4513645666654,1.6199851054554413,0.6388004876875932]    |
|26437.0|[0.048700964839629,1.4127042094810678,-1.0389207328646696E-5,-1.4141850644446894,0.0,0.0,0.7386950439500283,122.2291

In [10]:
# Randomly splits this :class:`DataFrame` with the provided weights.
train_data, test_data = scaled_df.randomSplit([.8,.2],seed=1234)

train_data.take(5)

[Row(load=16209.0, features=DenseVector([0.3258, -1.3755, 0.366, 1.366, 0.0, 0.0, 1.3866, 121.8546, 0.0, 0.1127])),
 Row(load=16675.0, features=DenseVector([0.3258, -1.3755, 1.2247, 0.7071, 0.0, 0.0, 1.3139, 121.8863, 0.7364, 0.2255])),
 Row(load=16709.0, features=DenseVector([-1.3754, -0.3314, 0.7071, 1.2247, 0.0, 0.0, 1.7196, 122.0324, 0.0, 0.3758])),
 Row(load=16946.0, features=DenseVector([0.6666, -1.2468, 0.7071, 1.2247, 0.0, 0.1664, 1.435, 121.4864, 0.0, 2.5928])),
 Row(load=17009.0, features=DenseVector([0.9452, -1.0518, 0.366, 1.366, 0.0, 0.0, 1.4471, 121.1943, 0.0, 0.977]))]

In [11]:
# Linear regression.
# The learning objective is to minimize the specified loss function, with regularization.
#More about regParam and elasticNetParam (seen as penalties on the loss function that minimises error in predicitons) can be found here: https://runawayhorse001.github.io/LearningApacheSpark/reg.html
# The maxIter parameter sets an upper limit on the number of iterations the optimization algorithm can perform regarding the loss function
# By default the LinearRegression function expects another column named "features"

lr = LinearRegression(labelCol="load", maxIter=500)

# Fits a model to the input dataset with optional parameters.
linearModel = lr.fit(train_data)

In [12]:
# Transform  is used to perform value predictions using the trained model
predicted = linearModel.transform(test_data)
predicted.printSchema()

predicted.select("load","features","prediction").show()

root
 |-- load: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- prediction: double (nullable = false)

+-------+--------------------+------------------+
|   load|            features|        prediction|
+-------+--------------------+------------------+
|16216.0|[0.77141735209955...| 22068.06741807801|
|16826.0|[0.32583482286144...|21137.598146455763|
|17352.0|[-0.9808868570328...| 20643.40006582645|
|17364.0|[-0.4889947446018...| 20674.85523010025|
|17379.0|[0.18214735722752...|20219.906228724216|
|17396.0|[-1.3255362000285...|21357.664270566158|
|17495.0|[-0.1579896063647...| 21126.03854991275|
|17708.0|[-1.0649069797603...|20066.880084899847|
|17850.0|[-0.9631904884983...|21890.241968869013|
|17850.0|[0.94519323616894...| 23837.97097647786|
|17896.0|[-0.7301528247730...| 20915.24371877881|
|17983.0|[0.18214735722752...| 21068.31012799561|
|17996.0|[0.92693112836009...|23676.099723102532|
|18010.0|[1.03219724640567...|23280.410296724294|
|18070.0|[-1.414715837442

In [13]:
# regression Summary
trainingSummary = linearModel.summary

print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)
print("Adjur2: %f" % trainingSummary.r2adj)
print("MSE: %f" % trainingSummary.meanSquaredError)
print("MAE: %f" % trainingSummary.meanAbsoluteError)

RMSE: 3560.167659
r2: 0.527683
Adjur2: 0.527459
MSE: 12674793.760845
MAE: 2876.655052
